In [23]:
from pathlib import Path
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from src.data import cbk_data

## Cleaning and exporting the data

We hve four sources of data that we need to clean and consolidate into one to be added in an posibble s3 file. 

In [24]:
files_types = {"cbk" : "csv", "fred" : "csv"}
# files_types = {"cbk" : "csv", "fred" : "csv", "m" : "xls", "x" : "xls"}

We read the data

In [25]:
raw_data = {}
for k, v in files_types.items():
    if v == "xls":
        raw_data[k] = pd.read_excel(Path.cwd().parent.joinpath("data", "raw_" + k + "." + v))
    else:
        raw_data[k] = pd.read_csv(Path.cwd().parent.joinpath("data", "raw_" + k + "." + v))

We explore the data

In [36]:
raw_data["cbk"].query("var == 'imae'")

,cod,date,value,var
692,87703,1991-01-31T00:00:00-06:00,34.542499,imae
693,87703,1991-02-28T00:00:00-06:00,31.288554,imae
694,87703,1991-03-31T00:00:00-06:00,29.636941,imae
695,87703,1991-04-30T00:00:00-06:00,29.691207,imae
696,87703,1991-05-31T00:00:00-06:00,30.123140,imae
...,...,...,...,...
1068,87703,2022-05-31T00:00:00-06:00,108.899486,imae
1069,87703,2022-06-30T00:00:00-06:00,109.098701,imae
1070,87703,2022-07-31T00:00:00-06:00,110.063999,imae
1071,87703,2022-08-31T00:00:00-06:00,110.343499,imae


In [38]:
raw_data["fred"]

,date,value,var
0,1986-01-02,25.56,crude_oil_wti
1,1986-01-03,26.00,crude_oil_wti
2,1986-01-06,26.53,crude_oil_wti
3,1986-01-07,25.85,crude_oil_wti
4,1986-01-08,25.87,crude_oil_wti
...,...,...,...
106187,2022-05-01,340385.00,usa_m
106188,2022-06-01,339642.00,usa_m
106189,2022-07-01,330040.00,usa_m
106190,2022-08-01,326472.00,usa_m


Let's make the date group by month and year. As the data for X and M are available since 2000, we filter all the vars to that year

In [39]:
grouped_data = {}
for df in raw_data:
       grouped_data[df] = (raw_data[df].assign(year = lambda x: pd.DatetimeIndex(x["date"]).year,
                                                    month = lambda x: pd.DatetimeIndex(x["date"]).month)
                                   #     .query("year >= 1999 & year <= 2022")
                                       .groupby(["year", "month", "var"])
                                       .agg({"value" : np.mean})               
                                       .reset_index())
                 

The cbk data for Exports and Imports is accumalted by month, we need to transform that to not accumulative value. So we extract those vars and transform them.

In [40]:
x_data = grouped_data["cbk"].query("var == 'exp'")
m_data = grouped_data["cbk"].query("var == 'imp'")
flowsvars = ["exp", "imp"]
grouped_data2 = grouped_data.copy()
grouped_data2["cbk"] = grouped_data["cbk"].query("var not in @flowsvars")

Transforming data in X-M

In [41]:
x_data2 = cbk_data.xm_notacc(x_data)
m_data2 = cbk_data.xm_notacc(m_data)

c:\Documentos\proyectos\xm4ecast\notebooks\..\src\data\cbk_data.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "value_lag"] = df.groupby("year")["value"].shift()
c:\Documentos\proyectos\xm4ecast\notebooks\..\src\data\cbk_data.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "value_lag"] = df.loc[:, "value_lag"].replace({np.nan: 0})
c:\Documentos\proyectos\xm4ecast\notebooks\..\src\data\cbk_data.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

Now, we join the dataset

In [42]:
final_dataset=pd.concat(grouped_data2, axis=0, ignore_index=True)
final_dataset=pd.concat([final_dataset, x_data2, m_data2], axis=0, ignore_index=True)

In [43]:
final_dataset.head()

,year,month,var,value
0,1991,1,imae,34.542499
1,1991,2,imae,31.288554
2,1991,3,imae,29.636941
3,1991,4,imae,29.691207
4,1991,5,imae,30.123140


In [48]:
final_dataset.groupby('var').agg({'year' : [np.min, np.max]})

year      
                        amin  amax
var                               
chn_m                   1992  2022
chn_x                   1992  2022
crude_oil_brent         1987  2022
crude_oil_wti           1986  2022
exp                     1994  2022
gas_henry_hub           1997  2022
gold                    2004  2022
imae                    1991  2022
imp                     1994  2022
mil_count               2009  2022
monex_amount_mean       2007  2022
nominal_broad_usdollar  2006  2022
tc_buy                  2006  2022
tc_monex                2007  2022
tc_sell                 2006  2022
tpm                     2006  2022
treasury1year           1959  2022
treasury3month          1954  2022
treasury4week           2001  2022
treasury6month          1958  2022
ue_m                    1990  2022
ue_x                    1990  2022
usa_m                   1992  2022
usa_x                   1992  2022
usdollar_euro           1999  2022
yuan_dollar_spot_tc     1981  2022

Aca decimos filtrar los años

In [51]:
final_dataset = final_dataset.query("year >= 1994 & year <= 2022")

Saving

In [22]:
with Path.cwd().parent.joinpath("data", "cleaned_data.csv") as e:
    final_dataset.to_csv(e, index=False)